In [9]:
import gymnasium as gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm
from torch import nn

import torch as T
from torch import optim
import torch.nn.functional as F

In [2]:
# print gym environment information
env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode="rgb_array"
)
print('observation space:', env.observation_space)
print('action space:', env.action_space)


observation space: Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)
action space: Discrete(4)


https://www.gymlibrary.dev/environments/box2d/lunar_lander/

## Planning:
The goal is to use DQN to solve the lunar lander problem.

Currently I am unclear on the following:
Exactly how to set up the network to receive inputs and outputs from the system
What kind of loss function should be used in this setup to make sure that it is learning correctly.
How to account for the various kinds of things can be observed in the observation space.

## approach
A good first step would be to try and experiment with basic approaches messing around with the system to get a grasp of it.
Then I'd like to experiment with a basic neural network feeding in maybe 2 of the parameters.

Edit: I think I will try to use the 4 parameters as inputs to the network and see how that goes.
Playing with the network structure and the loss function should be the next step.
Messing around trying to solve it by hand will probably not be very useful.



In [3]:
def create_animation(images):
    fig = plt.figure()
    ims = []

    for image in images:
        im = plt.imshow(image, animated=True);
        ims.append([im])
    plt.close()

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000);

    return ani

In [11]:

# reset environment
state = env.reset()
done = False
images = []
count = 0
total_reward = 0
while not done:
    count +=1
    # render environment
    image = env.render()
    images.append(image)
    # sample random action
    action = env.action_space.sample()
    # take action
    observation, state, reward, done, info = env.step(action)
    total_reward += reward

In [ ]:
class network(nn.module):
    def __init__(self, input_dims, fc1_dims=64, fc2_dims=64, n_actions=4, alpha=0.0001):
        super(network, self).__init__()

        self.fc1 = nn.Linear(input_dims, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.pi = nn.Linear(fc2_dims, n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
 
        self.to(self.device)
    
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        pi = T.softmax(self.pi(x), dim=1)

        return pi
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))